<a href="https://colab.research.google.com/github/Odoog/DLhomeworks/blob/master/JobsAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [11]:
!pip3 install scv

     |████████████████████████████████| 7.2MB 7.5MB/s 
     |████████████████████████████████| 348kB 38.3MB/s 


In [147]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from babel import Locale
locale = Locale('en', 'US')

CURRENCY_SYMBOLS = [''.join(filter(lambda x: not x.isalpha(), st)) for st in list(locale.currency_symbols.values())]

print(CURRENCY_SYMBOLS)

INDUSTRY_NAMES = conf('INDUSTRY_NAMES')

['$', '$', '$', '¥', '€', '£', '$', '₪', '₹', '¥', '₩', '$', '$', '$', '$', '₫', '', '$', '', '', '¤']


## Import

In [90]:
from __future__ import print_function
import os
import numpy as np
from google.colab import drive
from tqdm.autonotebook import tqdm
import pandas as pd

import requests as rq
from datetime import datetime
import csv
import json

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """


In [7]:
def driveReconnect():
  drive.mount('/content/drive')
  os.chdir('/content/drive/My Drive/MCH/data')
  os.listdir()

driveReconnect()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Library

In [5]:
def log(toLog = "", status = "log"):
    lg = open("log.txt", "a")
    currentTime = datetime.now().strftime("%H:%M:%S")
    errorSymbol = '!'
    rTimes = 0
    if status == "err":
        rTimes = 10
    lg.write(errorSymbol * rTimes + currentTime + " : " + toLog + "\n")

In [146]:
def conf(name, value = None):
    if value is not None:
        confFile = open('./configData.json', 'r')
        config = json.loads(confFile.read())
        confFile.close()

        config.update({name : value})

        confFile = open('./configData.json', 'w')
        confFile.write(json.dumps(config))
        confFile.close()

        return value
    else:
        confFile = open('./configData.json', 'r')
        config = json.loads(confFile.read())
        confFile.close()

        return config.get(name, None)

In [137]:
def getNetologyCourseInfo(url):
    if url == None:
        return url
    
    cName = url.split('/')[-1]

    rs = rq.get("https://netology.ru/backend/api/page_contents/programs%2F" + cName)

    courseInfo = []

    if rs.status_code == 200:
        try:
            courseInfo = rs.json()
        except:
            log('rs.content is ' + str(rs.content), 'err')    
    else:
        log('re.status_code is ' + str(rs.status_code), 'err')
        return

    skills = []

    try:
        skills = courseInfo["content"]["resume"]["skills"]
        return list(map(getPureSkillFromText, skills))
    except:
        try:
            skills = [t["title"] for t in courseInfo["content"]["resume"]["technologies"]]
            return skills
        except:
            return []

In [95]:
def updateNetologyCoursesData():
    rs = rq.get("https://netology.ru/backend/api/programs")

    courses = []

    if rs.status_code == 200:
        courses = rs.json()
    else:
        log('re.status_code is ' + str(rs.status_code), 'err')
        return

    with open('./Datasets/NetologyCoursesDataset.csv', 'w', encoding='UTF8') as f:
        writer = csv.writer(f)

        header = ['id', 'type', 'name', 'url', 'duration', 'projects_amount', 'date', 'main_direction', 'directions', 'skills', 'price_type', 'description']
        writer.writerow(header)

        for course in tqdm(courses):
            writer.writerow(
                [course.get(key, None) if key != 'skills' else getNetologyCourseInfo(course.get('url')) for key in header]
            )

    display(pd.read_csv('./Datasets/NetologyCoursesDataset.csv').head())

updateNetologyCoursesData()

,id,type,name,url,duration,projects_amount,date,main_direction,directions,skills,price_type,description
0,21275,profession,Тестировщик ПО,//netology.ru/programs/qa,7 месяцев,10 занятий,2021-07-01,"{'slug': 'development', 'name': 'Программирова...","[{'slug': 'development', 'name': 'Программиров...","['Java', 'GitHub', 'Git', 'Selenium', 'SQL', '...",paid,"<p>Станьте IT-специалистом, отвечающим за каче..."
1,21295,profession,Инженер по тестированию: с нуля до middle,//netology.ru/programs/qa-middle,14 месяцев,10 занятий,2021-07-01,"{'slug': 'development', 'name': 'Программирова...","[{'slug': 'development', 'name': 'Программиров...","['<p>Git</p>', '<p>Java</p>', '<p>Selenium</p>...",paid,"<p>Станьте IT-специалистом, отвечающим за каче..."
2,20528,profession,Интернет-маркетолог: с нуля до middle,//netology.ru/programs/distance-course-interne...,10 месяцев,61 занятие,2021-06-28,"{'slug': 'marketing', 'name': 'Маркетинг'}","[{'slug': 'marketing', 'name': 'Маркетинг'}]",[],paid,"<p>Основы маркетинга, анализ и сегментация цел..."
3,19718,profession,Python-разработчик с нуля,//netology.ru/programs/python,8 месяцев,26 занятий,2021-06-15,"{'slug': 'development', 'name': 'Программирова...","[{'slug': 'development', 'name': 'Программиров...","['<font color=""#000000"">Python</font>', '<p><f...",paid,"<p>Став Python-разработчиком, вы делаете первы..."
4,12701,profession,SMM-менеджер: продвижение бизнеса в социальных...,//netology.ru/programs/smm-specialist,8 месяцев,128 занятий,2021-06-28,"{'slug': 'marketing', 'name': 'Маркетинг'}","[{'slug': 'marketing', 'name': 'Маркетинг'}]","['Popsters', 'JagaJam', 'Target Hunter', 'Pepp...",paid,"<p>Социальные сети для бизнеса, построение стр..."


In [42]:
def getSkillsFromText(text):
    return []

In [191]:
def getPriceFromText(text):
    
    text = list(str(text))

    price = "".join(filter(lambda x: x.isdecimal() or x in CURRENCY_SYMBOLS or x in ['.', ','], text))

    if len(price) > 0:
        if price[0] == '.' or price[0] == ',':
            price = price[1:]
        else:
            if price[-1] == '.' or price[-1] == ',':
                price = price[:-1]
    return price

In [184]:
def getHighlightsFromText(text):
    return ""

In [ ]:
def getPureSkillFromText(text):
    return text

## Datasets manipulation


~load

In [161]:
jobPostingsDatasetNaukriClean = pd.read_csv('./Datasets/JobPostingsDatasetClean(Naukri.com).csv')
jobPostingsDatasetNaukriClean.head(1)

,Uniq Id,Crawl Timestamp,Job Title,Job Salary,Job Experience Required,Key Skills,Role Category,Location,Functional Area,Industry,Role
0,9be62c49a0b7ebe982a4af1edaa7bc5f,2019-07-05 01:46:07 +0000,Digital Media Planner,Not Disclosed by Recruiter,5 - 10 yrs,Media Planning| Digital Media,Advertising,Mumbai,"Marketing , Advertising , MR , PR , Media Plan...","Advertising, PR, MR, Event Management",Media Planning Executive/Manager


In [162]:
jobPostingsDatasetReedUk = pd.read_csv('./Datasets/JobPostingsDataset(Reed.uk).csv')
jobPostingsDatasetReedUk.head(1)

,category,city,company_name,geo,job_board,job_description,job_requirements,job_title,job_type,post_date,salary_offered,state
0,catering jobs,Chudleigh,Haulfryn Group,uk,reed,Apply now New opportunity not to be missed! H...,NaN,Commis Chef,"Permanent, full-time",3/12/2018,"£18,323 per annum",Devon


In [163]:
jobPostingsDatasetNaukriMessy = pd.read_csv('./Datasets/JobPostingsDatasetMessy(Naukri.com).csv')
jobPostingsDatasetNaukriMessy.head(1)

,company,education,experience,industry,jobdescription,jobid,joblocation_address,jobtitle,numberofpositions,payrate,postdate,site_name,skills,uniq_id
0,MM Media Pvt Ltd,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,0 - 1 yrs,Media / Entertainment / Internet,Job Description Send me Jobs like this Quali...,210516002263,Chennai,Walkin Data Entry Operator (night Shift),NaN,"1,50,000 - 2,25,000 P.A",2016-05-21 19:30:00 +0000,NaN,ITES,43b19632647068535437c774b6ca6cf8


In [164]:
globalCoursesInfoDataset = pd.read_csv('./Datasets/CoursesInfoDataset(CourseTalk.com).csv')
globalCoursesInfoDataset.head(1)

,CourseTitle,Description,Availability,Cost
0,C# Basics,In this course you will gain all the knowledge...,On demand,$25/mo


In [165]:
courseraGSIDataset = pd.read_csv('./Datasets/CourseraGSI.csv')
courseraGSIDataset.head(1)

,country,iso3,region,incomegroup,iso,competency_id,percentile_rank,percentile_category
0,United Arab Emirates,ARE,Middle East & North Africa,High income,AE,statistical-programming,0.864407,Cutting-edge (75%-100%)


In [169]:
skillsIndustryNeedsDatasetWorldBankGroup = pd.read_csv('./Datasets/IndustrySkillsNeeds(WorldBankGroup).csv', delimiter=';', encoding = 'latin-1')
skillsIndustryNeedsDatasetWorldBankGroup.head(20)

INDUSTRY_NAMES = conf("INDUSTRY_NAMES", list(set(skillsIndustryNeedsDatasetWorldBankGroup['industry_name'])))

In [170]:
skillsPenetrationDatasetWorldBankGroup = pd.read_csv('./Datasets/UseSkillPenetration(World Bank Group).csv', delimiter=';', encoding = 'latin-1')
skillsPenetrationDatasetWorldBankGroup.head(1)

,skill_group_category,skill_group_name,isic_section_index,isic_section_name,industry_name,skill_group_penetration_rate
0,Business Skills,Accounts Payable,M,Professional scientific and technical activities,Accounting,"0,00719"


~set structure

In [176]:
jobPostingsDatasets = {
    "naukri1" : jobPostingsDatasetNaukriClean,
    "reedUk" : jobPostingsDatasetReedUk,
    "naukri2" : jobPostingsDatasetNaukriMessy
}

~modify

In [189]:
jobPostingsDatasetNaukriClean.head(1)

,Uniq Id,Crawl Timestamp,Job Title,Job Salary,Job Experience Required,Key Skills,Role Category,Location,Functional Area,Industry,Role
0,9be62c49a0b7ebe982a4af1edaa7bc5f,2019-07-05 01:46:07 +0000,Digital Media Planner,Not Disclosed by Recruiter,5 - 10 yrs,Media Planning| Digital Media,Advertising,Mumbai,"Marketing , Advertising , MR , PR , Media Plan...","Advertising, PR, MR, Event Management",Media Planning Executive/Manager


In [195]:
jobPostings1 = jobPostingsDatasetNaukriClean
jobPostings1 = jobPostings1[['Job Title', 'Key Skills', 'Job Salary', 'Functional Area', 'Role Category']]

jobPostings1['Job Salary'] = list(map(getPriceFromText, jobPostings1['Job Salary']))
jobPostings1.rename(columns={"Job Salary": "salary"}, inplace=True)

jobPostings1.rename(columns={"Functional Area": "description_highlights"}, inplace=True)

jobPostings1.rename(columns={"Key Skills": "skills"}, inplace=True)

jobPostings1.rename(columns={"Role Category": "industry"}, inplace=True)

jobPostings1.rename(columns={"Job Title": "job_title"}, inplace=True)

jobPostings1.head(1)

,job_title,skills,salary,description_highlights,industry
0,Digital Media Planner,Media Planning| Digital Media,,"Marketing , Advertising , MR , PR , Media Plan...",Advertising


In [193]:
jobPostingsDatasetReedUk.head(1)

,category,city,company_name,geo,job_board,job_description,job_requirements,job_title,job_type,post_date,salary_offered,state
0,catering jobs,Chudleigh,Haulfryn Group,uk,reed,Apply now New opportunity not to be missed! H...,NaN,Commis Chef,"Permanent, full-time",3/12/2018,"£18,323 per annum",Devon


In [196]:
jobPostings2 = jobPostingsDatasetReedUk
jobPostings2 = jobPostings2[['category', 'job_title', 'job_description', 'salary_offered', 'job_requirements']]

jobPostings2['salary_offered'] = list(map(getPriceFromText, jobPostings2['salary_offered']))
jobPostings2.rename(columns={"salary_offered": "salary"}, inplace=True)

jobPostings2['job_description'] = list(map(getHighlightsFromText, jobPostings2['job_description']))
jobPostings2.rename(columns={"job_description": "description_highlights"}, inplace=True)

jobPostings2.rename(columns={"job_requirements": "skills"}, inplace=True)

jobPostings2.rename(columns={"category": "industry"}, inplace=True)

jobPostings2.head(1)

,industry,job_title,description_highlights,salary,skills
0,catering jobs,Commis Chef,,"£18,323",NaN


In [197]:
jobPostingsDatasetNaukriMessy.head(1)

,company,education,experience,industry,jobdescription,jobid,joblocation_address,jobtitle,numberofpositions,payrate,postdate,site_name,skills,uniq_id
0,MM Media Pvt Ltd,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,0 - 1 yrs,Media / Entertainment / Internet,Job Description Send me Jobs like this Quali...,210516002263,Chennai,Walkin Data Entry Operator (night Shift),NaN,"1,50,000 - 2,25,000 P.A",2016-05-21 19:30:00 +0000,NaN,ITES,43b19632647068535437c774b6ca6cf8


In [198]:
jobPostings3 = jobPostingsDatasetNaukriMessy
jobPostings3 = jobPostings3[['jobtitle', 'jobdescription', 'payrate', 'skills', 'industry']]

jobPostings3['payrate'] = list(map(getPriceFromText, jobPostings3['payrate']))
jobPostings3.rename(columns={"payrate": "salary"}, inplace=True)

jobPostings3['jobdescription'] = list(map(getHighlightsFromText, jobPostings3['jobdescription']))
jobPostings3.rename(columns={
    "jobdescription": "description_highlights",
    "jobtitle": "job_title"
    }, inplace=True)

jobPostings3.head(1)

,job_title,description_highlights,salary,skills,industry
0,Walkin Data Entry Operator (night Shift),,"1,50,0002,25,000",ITES,Media / Entertainment / Internet


~concat

In [199]:
jobPostings = pd.concat([jobPostings1, jobPostings2, jobPostings3], axis=0)

log("JobPosting1 shape is " + str(jobPostings1.shape))
log("JobPosting2 shape is " + str(jobPostings2.shape))
log("JobPosting3 shape is " + str(jobPostings3.shape))

log("JobPostings shape is " + str(jobPostings.shape))

In [201]:
jobPostingsFull = jobPostings.dropna(subset=['skills'])

log("JobPostings shape is " + str(jobPostingsFull.shape))

jobPostingsFull.head(30)

,job_title,skills,salary,description_highlights,industry
0,Digital Media Planner,Media Planning| Digital Media,,"Marketing , Advertising , MR , PR , Media Plan...",Advertising
1,Online Bidding Executive,pre sales| closing| software knowledge| clien...,,"Sales , Retail , Business Development",Retail Sales
2,Trainee Research/ Research Executive- Hi- Tec...,Computer science| Fabrication| Quality check|...,,"Engineering Design , R&D",R&D
3,Technical Support,Technical Support,"2,00,0004,00,000","IT Software - Application Programming , Mainte...",Admin/Maintenance/Security/Datawarehousing
4,Software Test Engineer -hyderabad,manual testing| test engineering| test cases|...,,IT Software - QA & Testing,Programming & Design
5,Opening For Adobe Analytics Specialist,adobe experience manager| digital| digital ma...,"7,00,00017,00,000","IT Software - Application Programming , Mainte...",Programming & Design
6,Sales- Fresher-for Leading Property Consultant,channel partners| real estate| negotiation| p...,"2,00,0003,00,000","Sales , Retail , Business Development",Retail Sales
7,Opportunity For Azure Devops Architect For Hy...,TFS| Azure| Git| VSTS| Docker| DynaTrace| Spl...,,"IT Software - Application Programming , Mainte...",Programming & Design
8,BDE- PUNE,Bde,,"Sales , Retail , Business Development",Institutional Sales
9,Technical Support/ Product Support,technical support| support services| applicat...,,"ITES , BPO , KPO , LPO , Customer Service , Op...",Voice
